In [27]:

from math import log

from Phase_1.src.Utils.SimplePositionalIndex import SimplePositionalIndex
from Phase_1.src.Utils.utilities import read_file

docs_url, docs_title, docs_content = read_file()


class DevelopedPositionalIndex(SimplePositionalIndex):
    def __init__(self, documents_url, documents_title, documents_content):
        super().__init__(documents_url, documents_title, documents_content, False)
        self.total_number_of_documents = len(documents_url)
        self.build_updated_positional_index()

    def build_updated_positional_index(self):
        for WORD in self.positional_index_structure.keys():
            number_of_unique_occurrences = len(self.positional_index_structure[WORD]['indexes'])
            for TERM_URL, DICTIONARY in self.positional_index_structure[WORD]['indexes'].items():
                DICTIONARY['tf idf'] = (1 + log(DICTIONARY['number of occurrences in document'])) * log(
                    self.total_number_of_documents / number_of_unique_occurrences)


pos_index = DevelopedPositionalIndex(documents_url=docs_url, documents_title=docs_title, documents_content=docs_content)


In [32]:
for word in pos_index.positional_index_structure.keys():
    for doc_url, dictionary in pos_index.positional_index_structure[word]['indexes'].items():
        print(dictionary['tf idf'])

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



8.310742863043687
9.409355151711798
5.477529518987472
5.477529518987472
5.477529518987472
5.477529518987472
5.477529518987472
5.477529518987472
5.477529518987472
5.477529518987472
5.477529518987472
5.477529518987472
5.477529518987472
5.477529518987472
5.477529518987472
9.274263661507513
5.477529518987472
5.477529518987472
5.477529518987472
5.477529518987472
9.274263661507513
5.477529518987472
5.477529518987472
5.477529518987472
5.477529518987472
5.477529518987472
5.477529518987472
5.477529518987472
5.477529518987472
5.477529518987472
5.477529518987472
5.477529518987472
5.477529518987472
5.477529518987472
5.477529518987472
5.477529518987472
5.477529518987472
5.477529518987472
5.477529518987472
5.477529518987472
5.477529518987472
5.477529518987472
5.477529518987472
5.477529518987472
9.274263661507513
9.274263661507513
5.477529518987472
5.477529518987472
5.477529518987472
5.477529518987472
5.477529518987472
5.477529518987472
5.477529518987472
9.409355151711798
4.845006960243961
4.84500696